In [1]:
from accidents_montreal import fetch_accidents_montreal,\
                               extract_accidents_montreal_dataframe
from road_network import fetch_road_network, extract_road_segments_DF
from weather import get_weather
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import atan2, sqrt, row_number, cos, sin, radians,\
                                  col, rank, avg
from pyspark.sql import functions, types, Row

In [2]:
def init_spark():
    return (SparkSession
            .builder
            .getOrCreate())


# init spark
spark = init_spark()

# retrieve datasets
fetch_accidents_montreal()
accidents_df = extract_accidents_montreal_dataframe(spark)

Skip fetching montreal accidents dataset: already downloaded
Skip extraction of accidents montreal dataframe: already done, reading from file


In [3]:
from pyspark.sql.functions import udf
import numpy as np



def extract_date_val(i):
    return udf(lambda val : val.split('/')[i])

@udf
def extract_hour(val):
    return val.split('-')[0].split(':')[0]

test = accidents_df.select('DT_ACCDN','LOC_LAT', 'LOC_LONG', 'HEURE_ACCDN')  \
                            .withColumn("year", extract_date_val(0)(accidents_df.DT_ACCDN)) \
                            .withColumn("month", extract_date_val(1)(accidents_df.DT_ACCDN)) \
                            .withColumn("day", extract_date_val(2)(accidents_df.DT_ACCDN)) \
                            .withColumn("HEURE_ACCDN", extract_hour(accidents_df.HEURE_ACCDN)) \
                            .drop('DT_ACCDN') \
                            .replace('Non précisé', '00')

def ai(row):
    try:
        return get_weather(int(row.LOC_LAT), 
                                 int(row.LOC_LONG), 
                                 int(row.year), 
                                 int(row.month), 
                                 int(row.day), 
                                 int(row.HEURE_ACCDN))
    except:
        print('error occured')
        return row

test2 = spark.createDataFrame(test.limit(1).collect(), test.columns)
test2 = test2.rdd \
    .map(lambda row: ai(row)) 

"""
# Test pour un seul element
row = test.sample(0.1).first()
row = get_weather(int(row.LOC_LAT), 
            int(row.LOC_LONG), 
            int(row.year), 
            int(row.month), 
            int(row.day), 
            int(row.HEURE_ACCDN))
print(row)"""

'\n# Test pour un seul element\nrow = test.sample(0.1).first()\nrow = get_weather(int(row.LOC_LAT), \n            int(row.LOC_LONG), \n            int(row.year), \n            int(row.month), \n            int(row.day), \n            int(row.HEURE_ACCDN))\nprint(row)'

In [4]:
import time
t=time.time()
print(test2.count())
t=time.time()-t
a=test2.first()

1


In [5]:
#list(a.asDict().keys())

In [6]:
it = test2.collect()
for i in it:
    print(i['Stn Press Flag'])
#

nan


In [7]:
#test3 = spark.createDataFrame(test2.collect(), list(a.asDict().keys()))
"""test4.write.parquet('data/test.parquet')"""

"test4.write.parquet('data/test.parquet')"

In [8]:
""""df=extract_accidents_montreal_dataframe(spark)
test=df.take(5)
test = list(map(lambda e: (e.LOC_LAT, e.LOC_LONG), test))
t= test[0]
lat, long = t
year, month, day = (2006,5,2)
hour = 0
print(get_weather(lat, long, year, month, day, hour))""""

SyntaxError: EOL while scanning string literal (<ipython-input-8-bcfd352dfc07>, line 8)

In [ ]:
"""import os
os.path.isfile('data/test.parquet')"""